In [2]:
import numpy as np
import qnm
from IMR.v5HM_BOB_generate_waveform_calibration import v5HM_BOB_generate_waveform_calibration as v5HM
from pyseobnr.generate_waveform import generate_modes_opt
import matplotlib.pyplot as plt
import kuibit
from kuibit.gw_mismatch import mismatch_from_strains
from kuibit.timeseries import TimeSeries

def brew_moonshine():
    M = 50
    f = 20
    Msol = 4.925491025543575903411922162094833998e-6
    Omega_0 = M*Msol*np.pi*f
    q = 1.5
    chi1 = 0.02
    chi2 = 0.01
    
    m1 = q/(1. + q)
    m2 = 1./(1. + q)
    ap = m1*chi1 + m2*chi2
    am = m1*chi1 - m2*chi2
    nu = m1*m2
    
    dSO = (
        -7.71251231383957 * am ** 3
        - 17.2294679794015 * am ** 2 * ap
        - 238.430383378296 * am ** 2 * nu
        + 69.5461667822545 * am ** 2
        - 10.5225438990315 * am * ap ** 2
        + 362.767393298729 * am * ap * nu
        - 85.8036338010274 * am * ap
        - 1254.66845939312 * am * nu ** 2
        + 472.431937787377 * am * nu
        - 39.742317057316 * am
        - 7.58458103577458 * ap ** 3
        - 42.7601129678844 * ap ** 2 * nu
        + 18.1783435552183 * ap ** 2
        - 201.905934468847 * ap * nu ** 2
        - 90.5790079104259 * ap * nu
        + 49.6299175121658 * ap
        + 478.546231305475 * nu ** 3
        + 679.521769948995 * nu ** 2
        - 177.334831768076 * nu
        - 37.6897780220529
    )
    
    para6 = np.array(
        [4.17877875e01, -3.02193382e03, 3.34144394e04, -1.69019140e05, 3.29523262e05]
    )
    a6 = para6[0] + para6[1] * nu + para6[2] * nu ** 2 + para6[3] * nu ** 3 + para6[4] * nu ** 4
    
    pardTNS = np.array(
        [1.00513217e01, -5.96231800e01, -1.05687385e03, -9.79317619e03, 5.55652392e04]
    )
    Deltat_NS = nu ** (-1.0 / 5 + pardTNS[0] * nu) * (
        pardTNS[1] + pardTNS[2] * nu + pardTNS[3] * nu ** 2 + pardTNS[4] * nu ** 3
    )
    
    Deltat_S = nu ** (-1.0 / 5 + 0 * nu) * (
        8.39238879807543 * am ** 2 * ap
        - 16.9056858928167 * am ** 2 * nu
        + 7.23410583477034 * am ** 2
        + 6.38975598319936 * am * ap ** 2
        + 179.569824846781 * am * ap * nu
        - 40.6063653476775 * am * ap
        + 144.253395844761 * am * nu ** 2
        - 90.1929138487509 * am * nu
        + 14.2203101910927 * am
        - 6.78913884987037 * ap ** 4
        + 5.39962303470497 * ap ** 3
        - 132.224950777226 * ap ** 2 * nu
        + 49.8016443361381 * ap ** 2
        + 384.201018794943 * ap * nu ** 2
        - 141.253181790353 * ap * nu
        + 17.5710132409988 * ap
    )
    
    Deltat_v5HM = Deltat_NS + Deltat_S
    t_FEOB, h22_FEOB = v5HM(M,q,chi1,chi2,f,a6,dSO,Deltat_v5HM,2.4627455127717882e-05,debug = False)
    t_v5HM, hlm_v5HM = generate_modes_opt(q,chi1,chi2,Omega_0,debug = False)
    h22_v5HM = hlm_v5HM['2,2']
    FEOB = TimeSeries(t_FEOB, h22_FEOB)
    SEOB = TimeSeries(t_v5HM,h22_v5HM)
    return FEOB, SEOB

FEOB, SEOB = brew_moonshine()

In [3]:
mismatch = mismatch_from_strains(FEOB,SEOB)
print(mismatch)

(0.002733075508517424, (3.5541250222429985, 0.3061224489795915))


In [ ]:
# plot the non-nqc-ed waveform
dT = 2.4627455127717882e-05/M/4.925490947641266978197229498498379006e-6
t_new = np.arange(dynamics_BOB[0,0],dynamics_BOB[-1,0],dT)
plt.plot(h22_v5HM[:,0], np.sqrt(h22_v5HM[:,1]**2 + h22_v5HM[:,2]**2),color = 'black',label = 'pyseobnr')
plt.plot(t_new - t_new[0] + h22_v5HM[0,0],np.abs(h22_BOB_noNQC),label = 'ours, no NQC')
#plt.plot(t_new_BOB - t_new_BOB[0] + h22_v5HM[0,0],np.real(h22_BOBcalibDeltat_noNQC),label = 'ours, BOB Deltat, no NQC')
plt.xlim(-150,50)
plt.legend()
plt.show()


In [ ]:
# plot the nqc-ed waveform
dT = 2.4627455127717882e-05/M/4.925490947641266978197229498498379006e-6
t_new = np.arange(dynamics_BOB[0,0],dynamics_BOB[-1,0],dT)
plt.plot(h22_v5HM[:,0], np.sqrt(h22_v5HM[:,1]**2 + h22_v5HM[:,2]**2),color = 'black',label = 'pyseobnr')
plt.plot(t_new - t_new[0] + h22_v5HM[0,0],np.abs(h22_BOB_NQC),label = 'ours, NQC')
plt.xlim(- 100,50)
plt.ylim(0,0.4)
plt.legend()
plt.show()


In [ ]:
# plot the full waveform amplitude
plt.plot(h22_v5HM[:,0], np.abs(h22_v5HM[:,1] + 1j*h22_v5HM[:,2]),color = 'black',label = 'pyseobnr')
plt.plot(h22_BOB_opt[:,0],np.abs(h22_BOB_opt[:,1] + 1j*h22_BOB_opt[:,2]),label = 'SEOBNRv5HM + BOB')
plt.xlim(-100,50)
plt.xlabel(r'$t(M)$',fontsize = 12)
plt.ylabel(r'$|h|$',fontsize = 12)
plt.legend()
plt.savefig(f"hamp_comparison_q_{qs}_chi1_{chi1s}_chi2_{chi2s}")
plt.show()


In [ ]:
# plot the full waveform phase

plt.plot(h22_v5HM[:,0], np.unwrap(np.angle(h22_v5HM[:,1]+1j*h22_v5HM[:,2])),color = 'black',label = 'pyseobnr')
plt.plot(h22_BOB_opt[:,0],np.unwrap(np.angle(h22_BOB_opt[:,1]+1j*h22_BOB_opt[:,2])),label = 'ours')
plt.xlim(-100,50)
#plt.ylim(1100,1300)
plt.legend()
plt.show()


In [ ]:
# plot h_+ (match phases at peak)
v5HM_amp = np.abs(h22_v5HM[:,1] - 1j*h22_v5HM[:,2])    
v5HM_phase = np.unwrap(np.angle(h22_v5HM[:,1] - 1j*h22_v5HM[:,2]))

BOB_amp = np.abs(h22_BOB_opt[:,1] - 1j*h22_BOB_opt[:,2])
BOB_phase = np.unwrap(np.angle(h22_BOB_opt[:,1] - 1j*h22_BOB_opt[:,2]))
BOB_idx_phase_match = np.argmin(np.abs(h22_BOB_opt[:,0]))

#BOBcalibDeltat_amp = np.abs(h22_BOBcalibDeltat_calib[:,1] - 1j*h22_BOBcalibDeltat_calib[:,2])
#BOBcalibDeltat_phase = np.unwrap(np.angle(h22_BOBcalibDeltat_calib[:,1] - 1j*h22_BOBcalibDeltat_calib[:,2]))
#BOBcalibDeltat_idx_phase_match = np.argmin(np.abs(h22_BOBcalibDeltat_calib[:,0]))


BOB_v5HM_phase_match = np.interp(h22_BOB_opt[:,0],h22_v5HM[:,0],v5HM_phase)[BOB_idx_phase_match]
BOB_phase = BOB_phase - BOB_phase[BOB_idx_phase_match] + BOB_v5HM_phase_match

#BOBcalibDeltat_v5HM_phase_match = np.interp(h22_BOBcalibDeltat_calib[:,0],h22_v5HM[:,0],v5HM_phase)[BOBcalibDeltat_idx_phase_match]
#BOBcalibDeltat_phase = BOBcalibDeltat_phase - BOBcalibDeltat_phase[BOBcalibDeltat_idx_phase_match] + BOBcalibDeltat_v5HM_phase_match

h22_BOB_plus = BOB_amp*np.cos(BOB_phase)
h22_BOB_cross = BOB_amp*np.sin(-1*BOB_phase)

#h22_BOBcalibDeltat_plus = BOBcalibDeltat_amp*np.cos(BOBcalibDeltat_phase)
#h22_BOBcalibDeltat_cross = BOBcalibDeltat_amp*np.sin(-1*BOBcalibDeltat_phase)

plt.plot(h22_v5HM[:,0],h22_v5HM[:,1],color = 'black',label = 'pyseobnr')
plt.plot(h22_BOB_opt[:,0],h22_BOB_plus,label = 'SEOBNRv5HM + BOB')
#plt.plot(h22_BOB_calib[:,0],h22_BOB_calib[:,1],label = 'SEOBNRv5HM + BOB')
#plt.plot(h22_BOBcalibDeltat_calib[:,0],h22_BOBcalibDeltat_plus,label = 'ours, BOB Deltat')
plt.xlim(-100,50)
plt.xlabel(r'$t(M)$',fontsize = 12)
plt.ylabel(r'$h_{+}$',fontsize = 12)
plt.legend()
plt.savefig(f"hplus_comparison_q_{qs}_chi1_{chi1s}_chi2_{chi2s}")
plt.show()


In [ ]:
# plot h_x (match phases at peak)

plt.plot(h22_v5HM[:,0],h22_v5HM[:,2],color = 'black',label = 'pyseobnr')
plt.plot(h22_BOB_opt[:,0],h22_BOB_cross,label = 'SEOBNRv5HM + BOB')
#plt.plot(h22_BOBcalibDeltat_calib[:,0],h22_BOBcalibDeltat_cross,label = 'ours, BOB Deltat')
plt.xlim(-100,50)
plt.legend()
plt.xlabel(r'$t(M)$',fontsize = 12)
plt.ylabel(r'$h_{\times}$',fontsize = 12)
plt.savefig(f"hcross_comparison_q_{qs}_chi1_{chi1s}_chi2_{chi2s}")
plt.show()


In [ ]:
# Amplitude derivatives
BOB_damp = np.gradient(BOB_amp, h22_BOB_opt[:,0])
#BOBcalibDeltat_damp = np.gradient(BOBcalibDeltat_amp, h22_BOBcalibDeltat_calib[:,0])
v5HM_damp = np.gradient(v5HM_amp,h22_v5HM[:,0])
plt.plot(h22_v5HM[:,0],v5HM_damp,label = 'pySEOBNR',color = 'black')
plt.plot(h22_BOB_opt[:,0],BOB_damp,label = 'SEOBNRv5HM+BOB')
#plt.plot(h22_BOBcalibDeltat_calib[:,0],BOBcalibDeltat_damp,label = 'SEOBNRv5HM+BOB(calib Deltat)')
plt.xlim(-100,50)
plt.xlabel(r'$t(M)$',fontsize = 12)
plt.ylabel(r'$\dot{|h|}$',fontsize = 12)
plt.legend()
plt.savefig(f"hdamp_comparison_q_{qs}_chi1_{chi1s}_chi2_{chi2s}")
plt.show()


In [ ]:
BOB_ddamp = np.gradient(BOB_damp, h22_BOB_opt[:,0])
v5HM_ddamp = np.gradient(v5HM_damp,h22_v5HM[:,0])
plt.plot(h22_v5HM[:,0],v5HM_ddamp,label = 'pySEOBNR',color = 'black')
plt.plot(h22_BOB_opt[:,0],BOB_ddamp,label = 'SEOBNRv5HM+BOB')
plt.xlim(-100,50)
plt.xlabel(r'$t(M)$',fontsize = 12)
plt.ylabel(r'$\ddot{|h|}$',fontsize = 12)
plt.legend()
plt.tight_layout()
plt.savefig(f"hddamp_comparison_q_{qs}_chi1_{chi1s}_chi2_{chi2s}")
plt.show()


In [ ]:
# Phase derivatives

BOB_dphi = np.gradient(np.abs(BOB_phase), h22_BOB_opt[:,0])
v5HM_dphi = np.gradient(np.abs(v5HM_phase),h22_v5HM[:,0])
plt.plot(h22_v5HM[:,0],v5HM_dphi,label = 'pySEOBNR',color = 'black')
plt.plot(h22_BOB_opt[:,0],BOB_dphi,label = 'SEOBNRv5HM+BOB')
plt.xlim(-100,50)
plt.ylim(0,1)
plt.xlabel(r'$t(M)$',fontsize = 12)
plt.ylabel(r'$\omega$',fontsize = 12)
plt.legend()
plt.savefig(f"hfreq_comparison_q_{qs}_chi1_{chi1s}_chi2_{chi2s}")
plt.show()


In [ ]:
BOB_ddphi = np.gradient(BOB_dphi, h22_BOB_opt[:,0])
v5HM_ddphi = np.gradient(v5HM_dphi,h22_v5HM[:,0])
plt.plot(h22_v5HM[:,0],v5HM_ddphi,label = 'pySEOBNR',color = 'black')
plt.plot(h22_BOB_opt[:,0],BOB_ddphi,label = 'SEOBNRv5HM+BOB')
plt.xlim(-100,50)
plt.ylim(0,0.012)
plt.xlabel(r'$t(M)$',fontsize = 12)
plt.ylabel(r'$\dot{\omega}$',fontsize = 12)
plt.legend()
plt.tight_layout()
plt.savefig(f"hdfreq_comparison_q_{qs}_chi1_{chi1s}_chi2_{chi2s}")
plt.show()


In [ ]:
import grav_waldo.evaltoolkit as waldo_mm
import kuibit.timeseries as kuibit_ts
import kuibit.gw_mismatch as kuibit_mismatch

h22_v5HM_complex = h22_v5HM[:,1] - 1j*h22_v5HM[:,2]
h22_v5HM_ts = kuibit_ts.TimeSeries(h22_v5HM[:,0],h22_v5HM_complex)
h22_BOB_plus_new = np.interp(h22_v5HM[:,0],h22_BOB_opt[:,0],h22_BOB_plus)
h22_BOB_cross_new = np.interp(h22_v5HM[:,0],h22_BOB_opt[:,0],h22_BOB_cross)
h22_BOB_complex = h22_BOB_plus - 1j*h22_BOB_cross
h22_BOB_ts = kuibit_ts.TimeSeries(h22_BOB_opt[:,0],h22_BOB_complex)
h22_BOB_complex_new = h22_BOB_plus_new - 1j*h22_BOB_cross_new
#mm = waldo_mm.Mismatch(h22_v5HM_ts.t,h22_v5HM_ts.y,h22_BOB_complex_new)
mm = kuibit_mismatch.mismatch_from_strains(h22_v5HM_ts, h22_BOB_ts)
print(mm)